In [22]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from IPython.display import Video
import os
import numpy as np
import pandas as pd
import pickle
import subprocess
from tqdm import tqdm
from utils_text import (timestamp_to_sec, sec_to_timestamps, create_sentences_from_words, squash_timestamps, clean_script_for_prompt)

In [8]:
class VideoCutter:
    def __init__(self, sentences_list, video_path, transcription_path, save_path = 'result'):
        
        self.video_path = video_path
        self.sentences_list = sentences_list
        self.tmp_folder = save_path
        self.make_folders()
        
        with open(transcription_path, 'rb') as f:
            self.transcription_results = pickle.load(f)
            
        self.sentence_dict, self.subtitles_dict = create_sentences_from_words(self.transcription_results) 
        
    def make_folders(self):
        os.makedirs(f"{self.tmp_folder}", exist_ok=True)
        os.makedirs(f"{self.tmp_folder}/videos", exist_ok=True)
        os.makedirs(f"{self.tmp_folder}/concat_videos", exist_ok=True)
        
    def cut_one_video(self, video, start, end, output_name):

        cmd = f"ffmpeg -y -copyts -ss {start} -i '{video}' -threads {os.cpu_count()} -to {end} -map 0 '{output_name}' -loglevel error"

        subprocess.run(cmd, shell=True)


    def concat_videos(self, filename, output_name):

        cmd = f"ffmpeg -y -f concat -safe 0 -threads {os.cpu_count()} -i '{filename}' -c copy '{output_name}' -loglevel error"

        subprocess.run(cmd, shell=True)

    def cut_videos(self):
        print ('Cutting videos')
        for num, timestamp in tqdm(enumerate(self.render_times['timestamps'])):
            output_name = num
#             output_name = f"Short {int(self.output_dict['Ranking'][num][0])+1} {self.output_dict['Name'][num]}"
#             output_name = output_name.replace("'", '').replace('"', '').replace(':', '')

            output_name = f"{self.tmp_folder}'/videos/'{output_name}.mp4"

            if len(timestamp)>1:
                concat_list = []
                for count, cut in enumerate(timestamp):
                    try:
                        start = timestamp_to_sec(cut[0])
                        end = timestamp_to_sec(cut[1])
                    except:
                        start = cut[0]
                        end = cut[1]

                    output_name_tmp = f'concat_videos/{num}_{count}.mp4'      
                    concat_list.append(f"file {output_name_tmp}\n")
                    output_name_tmp = f'{self.tmp_folder}/concat_videos/{num}_{count}.mp4'      
                    self.cut_one_video(self.video_path, start, end, output_name_tmp)
                with open(f"{self.tmp_folder}/file.txt", "w") as output:
                    output.writelines(concat_list)
                self.concat_videos(f"{self.tmp_folder}/file.txt", output_name)
            else:
                try:
                    start = timestamp_to_sec(timestamp[0][0])
                    end = timestamp_to_sec(timestamp[0][1])
                except:
                    start = timestamp[0][0]
                    end = timestamp[0][1]
                self.cut_one_video(self.video_path, start, end, output_name)
                
    def process (self):
        
        df = clean_script_for_prompt(self.sentence_dict, self.transcription_results, self.tmp_folder)
        
        self.render_times = {"timestamps": [], "positions": []}

        for highlight in self.sentences_list:
            positions, new_timestamps = squash_timestamps(highlight, self.sentence_dict["timestamps"])
            self.render_times["positions"].append(positions)
            self.render_times["timestamps"].append(new_timestamps)
            
        with open(f"{self.tmp_folder}/render_times.pickle", "wb") as f:
            pickle.dump(self.render_times, f)
            
        self.cut_videos()


In [12]:
sentences = [[0, 1, 2, 9, 10], [13, 14, 15]]

# sentences = [[14,15,16]]

In [10]:
transcription_path = "files/transcription.pickle"
video_path = "files/downloaded_video.mp4"

In [13]:
videocutter = VideoCutter(sentences, video_path, transcription_path)

In [3]:
with open("files/transcription.pickle", 'rb') as f:
    transcription_results = pickle.load(f)

In [16]:
sentence_dict = videocutter.sentence_dict

In [17]:
diarization = transcription_results["diarization"]

In [135]:
df = clean_script_for_prompt(sentence_dict, diarization, ".")

In [136]:
df.head(20)

,index,speaker,sentence,length
0,0,SPEAKER_02,My first guest is the man who made electric ca...,15.70
1,1,SPEAKER_02,"And then on Tuesday, he's going to work on tha...",5.50
2,2,SPEAKER_02,He also tweets a lot.,1.30
3,3,SPEAKER_02,"Elon Musk, ladies and gentlemen.",1.72
4,4,SPEAKER_02,Look at you.,7.90
5,5,SPEAKER_00,Wow.,4.00
6,6,SPEAKER_00,Did I get the full order of things that you do...,7.00
7,7,SPEAKER_02,"When I was reading there, I left out the tunne...",3.28
8,8,SPEAKER_03,"Uh, yeah.",0.98
9,9,SPEAKER_03,A lot of jobs.,0.90


In [ ]:
videocutter.process()

In [63]:
videocutter.sentences_list

[[0, 1, 2, 9, 10], [13, 14, 15]]

In [59]:
videocutter.render_times

{'timestamps': [[[0.0, 22.5], [47.38, 49.96]], [[52.94, 56.24]]],
 'positions': [[[0, 2], [9, 10]], [[13, 15]]]}

In [5]:
Video("result/videos/0.mp4")